In [ ]:
from nbdev import *
%nbdev_default_export cell_data
%reload_ext autoreload
%autoreload 2

Cells will be exported to wtlike.cell_data,
unless a different module is specified after an export flag: `%nbdev_export special.module`


# Manage cell data

In [ ]:
%nbdev_export
import os
import numpy as np
import pandas as pd
from wtlike.config import *
from wtlike.source_data import *

In [ ]:
%nbdev_export
class CellData(SourceData):
    """Manage a set of cells generated from a data set
    
    """
    
    def __init__(self, config, source, bins=None):
        """
        Invoke superclass to load photon data and exposure for the source.
        Manage a list of cells
        """
        super().__init__(config, source, )

        self.source_name =source.name
        self.verbose = config.verbose
        self.use_uint8  = config.use_uint8
        photon_data = self.p_df
        
        # exposure binned as well
        self.fexposure, bins = self.binned_exposure( time_bins=bins, ) #bins, exposure)

        # manage bins 
        self.N = len(bins)-1 # number of bins
        self.bins = bins
        self.bin_centers = 0.5*(bins[1:]+bins[:-1])
        
        # restrict photons to range of bin times
        photons = photon_data.query(f'{bins[0]}<time<{bins[-1]}')

        # get the photon data with good weights, not NaN
        w = photons.weight
        good = np.logical_not(np.isnan(w))
        self.photons = photons.loc[good]
        self.weights = w = self.photons.weight.values
        
        # estimates for averate signal and background per cell
        self.S = np.sum(w)/self.N
        self.B = np.sum(1-w)/self.N

        # use photon times to get indices of bin edges
        self._edges = np.searchsorted(self.photons.time, bins)

    def __repr__(self):
        return f'''{self.__class__}:
        {len(self.fexposure)} intervals from {self.bins[0]:.1f} to {self.bins[-1]:.1f} for source {self.source_name}
        S {self.S:.2f}  B {self.B:.2f} '''

    def __getitem__(self, i):
        """ get info for ith time bin and return dict with
            t : MJD
            tw: bin width,
            e: exposure as fraction of total,
            n : number of photons in bin
            w : list of weights as uint8 integers<=255
            S,B:  value
        """
        k   = self._edges
        w = self.weights[k[i]:k[i+1]]
        wts = np.array(w*256, np.uint8) if self.use_uint8 else w
        n = len(wts)
        e = self.fexposure[i]
        tw  = self.bins[i+1]-self.bins[i]

        return dict(
                t=self.bin_centers[i], # time
                tw = tw,  # bin width
                e=e, # moving to this name
                n=n, # number of photons in bin
                w=wts,
                S= e *self.S,
                B= e *self.B,
                )

    def __len__(self):
        return self.N

    @property
    def dataframe(self):
        """ combine all cells into a dataframe
        """
        df = pd.DataFrame([cell for cell in self])
        return df

In [ ]:
%nbdev_hide
from nbdev.export import notebook2script
notebook2script()
!date

Converted 00_config.ipynb.
Converted 01_data_man.ipynb.
Converted 01_effective_area.ipynb.
Converted 02-source_data.ipynb.
Converted 02_gti.ipynb.
Converted 03_exposure.ipynb.
Converted 04_photon_data.ipynb.
Converted 05_weights.ipynb.
Converted 06_poisson.ipynb.
Converted 07_cell_data.ipynb.
Converted 07_cells.ipynb.
Converted 08_loglike.ipynb.
Converted 09_lightcurve.ipynb.
Converted 10_simulation.ipynb.
Converted 14_bayesian.ipynb.
Converted index.ipynb.
Mon Mar 29 17:17:53 PDT 2021
